In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [86]:
def read_csv(file):
    df = pd.read_csv(file, parse_dates=True, index_col=0)
    return df

In [87]:
"""
On Balance Volume (OBV)
Params: 
    data: pandas DataFrame
    close_col: the name of the CLOSE values column
    vol_col: the name of the VOL values column
    
Returns:
    copy of 'data' DataFrame with 'obv' column added
"""
def on_balance_volume(data, close_col='<CLOSE>', vol_col='<VOL>'):
    for index, row in data.iterrows():
        if index > 0:
            last_obv = data.at[index - 1, 'obv']
            if row[close_col] > data.at[index - 1, close_col]:
                current_obv = last_obv + row[vol_col]
            elif row[close_col] < data.at[index - 1, close_col]:
                current_obv = last_obv - row[vol_col]
            else:
                current_obv = last_obv
        else:
            last_obv = 0
            current_obv = row[vol_col]

        data.at[index, 'obv'] = current_obv
        
    
    return data

In [88]:
"""
Rate of Change (ROC) with respect to Small and Large time windows  
Params: 
    data: pandas DataFrame
    windowSmallSize: size of "SMALL" time window
    windowLargeSize: size of "LARGE" time window
    t_col: the name of the <TARGET> column at which the rate of change is calculated
    
Returns:
     pSmall: rate of change with respect to the "SMALL" time window
     pLarge: rate of change with respect to the "LARGE" time window
     pInd: sum of pSmall and pLarge (aggregated rate of change)
"""

def rate_of_change(data:pd.DataFrame = None, t_col = '<CLOSE>', windowSmallSize:int = 13, windowLargeSize:int = 52):
    v_data = data[t_col].values
    dim = len(v_data)
    pSmall = np.full(dim,np.nan)
    pLarge = np.full(dim,np.nan)
    pInd = np.full(dim,np.nan)
    last_nan_index = sum(np.isnan(v_data))
    for t in range(last_nan_index,dim):
        vtemp = v_data[last_nan_index:t+1]
        windowSmall = vtemp[-windowSmallSize:]
        lowest = min(windowSmall)
        pSmall[t] = (vtemp[-1]/lowest - 1) * 100
        windowLarge = vtemp[-windowLargeSize:]
        largest = max(windowLarge)
        pLarge[t] = (vtemp[-1]/largest - 1) * 100
    
    pInd =  pSmall + pLarge
    
    return pInd, pSmall, pLarge

In [89]:
""" Slope (derivative) - similar to "momentum indicator" 
Params: 
    data: pandas DataFrame
    t_col: DataFrame column at which the first derivative is calculated   
    lag = lag for the difference operator
Returns:
     diff: pandas series
"""
def slope(data:pd.DataFrame = None, t_col = '<CLOSE>', lag:int = 3):
    diff =  data[t_col].diff(lag)
    return diff


---

## Feature Engineering on weekly data:

Iterates through a list of DataFrames containing the _Exchanges_ with weekly granulation and calculates the features to each of the _Exchanges_.

Returns a list of DataFrames containing the features calculated for each _Exchange_. 

**inputs**:
<ul>
   <li> 'exchange_name_list:' : list of strings containng the name of the exchanges, ex: ['sp500', 'nasdaq', 'etc.']
   <li>  'exchange_dataframe_list': list of DataFrames containing pandas DataFrames correponding to each _exchange_ , ex: [v_sp500_weekly, v_nasdaq_weekly, etc.]
   <li> 'frequency' = 'weekly_' (string)  
</ul>

**output**:
<ul>
   <li> 'exchange_features_dataframe_list:' : list of DataFrames (with weekly granularity), each one containing the calculated features for each _Exchange_.
</ul>


In [90]:
def get_weekly_features_for_exchanges(df_to_be_engineered, exchange_name_list:list = []):
    
    # function output: a list of DataFrames containing the calculated features for each exchange
    exchange_features_dataframe_list = []
    df_initial = df_to_be_engineered
    for name in exchange_name_list:
        df_to_be_engineered = df_initial
        print("Feature Engineering for Exchange", name)
        filter_col = [col for col in df_to_be_engineered if col.startswith(name)]
        # get a copy of the current exchange dataframe
        v_data = df_to_be_engineered[filter_col]
        # get the current exchange name
        current_exchange_name = name
        # DataFrame that will hold the generated features
        v_data_features = pd.DataFrame(index = v_data.index)

        # ----------------------------------------------------------------------
        # closing price trend (from closing price time series) 

        # closing price
        #df_field = current_exchange_name +  '_close'
        #v_data_features[df_field] = v_data.loc[:,current_exchange_name+'_Close'].values 

        # exponential moving average of 5 weeks (trend)
        period = 5 # smoothing period
        df_field = current_exchange_name + '_close_ema_' + str(period)
        v_data_features[df_field] = v_data[current_exchange_name+'_Close'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # exponential moving average of 13 weeks (trend)
        period = 13 # smoothing period
        df_field = current_exchange_name + '_close_ema_' + str(period)
        v_data_features[df_field] = v_data[current_exchange_name+'_Close'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 

        # exponential moving average of 26 weeks (trend)
        period = 26 # smoothing period
        df_field = current_exchange_name +  '_close_ema_' + str(period)
        v_data_features[df_field] = v_data[current_exchange_name+'_Close'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # exponential moving average of 52 weeks (trend)
        period = 52 # smoothing period
        df_field = current_exchange_name +  '_close_ema_' + str(period)
        v_data_features[df_field] = v_data[current_exchange_name+'_Close'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 

        # ----------------------------------------------------------------------
        # Volatility, Skewness and Kurtosis (from log weekly return distribution)

        # compute the logarithmic returns using the closing price 
        v_logRet = np.log(v_data[current_exchange_name+'_Close'] / v_data[current_exchange_name+'_Close'].shift(1))
        df_field = current_exchange_name + '_log_returns'
        v_data_features[df_field] = v_logRet 

        # compute 5-week annualized volatility
        period = 5
        v_feature = v_logRet.rolling(window=period).std() * np.sqrt(52)
        df_field = current_exchange_name + 'volatility_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 

        # compute 13-week annualized volatility
        period = 13
        v_feature = v_logRet.rolling(window=period).std() * np.sqrt(52)
        df_field = current_exchange_name  +  '_volatility_' + str(period) 
        v_data_features[df_field] = v_feature 
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute 26-week annualized volatility
        period = 26
        v_feature = v_logRet.rolling(window=period).std() * np.sqrt(52)
        df_field = current_exchange_name +  '_volatility_' + str(period) 
        v_data_features[df_field] = v_feature 
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute 52-week annualized volatility
        period = 52
        v_feature = v_logRet.rolling(window=period).std() * np.sqrt(52)
        df_field = current_exchange_name  +  '_volatility_' + str(period) 
        v_data_features[df_field] = v_feature 
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        # compute slope changes (second derivative)
        t_col2 = df_field
        period = 1  
        order = 2  
        diff =  slope(v_data_features, t_col=t_col2, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 

        # compute 13-week skewness 
        period = 13
        v_feature = v_logRet.rolling(window=period).skew()
        df_field = current_exchange_name  + '_skewness_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute 26-week skewness 
        period = 26
        v_feature = v_logRet.rolling(window=period).skew()
        df_field = current_exchange_name +  '_skewness_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute 52-week skewness 
        period = 52
        v_feature = v_logRet.rolling(window=period).skew()
        df_field = current_exchange_name + '_skewness_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute 13-week kurtosis
        period = 13
        v_feature = v_logRet.rolling(window=period).kurt()
        df_field = current_exchange_name + '_kurtosis_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute 26-week kurtosis
        period = 26
        v_feature = v_logRet.rolling(window=period).kurt()
        df_field = current_exchange_name + '_urtosis_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
       
        # compute 52-week kurtosis
        period = 52
        v_feature = v_logRet.rolling(window=period).kurt()
        df_field = current_exchange_name + '_kurtosis_' + str(period) 
        v_data_features[df_field] = v_feature
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # ----------------------------------------------------------------------
        # Volume Variation (from daily closing price and volume)

        # reset the index of the DataFrame, and use the default one (int) instead
        v_data_cp = v_data.reset_index(level=None, drop=True, inplace=False)

        # compute the on balance volume (OBV) indicator 
        df = on_balance_volume(v_data_cp, close_col=current_exchange_name+'_Close', vol_col=current_exchange_name+'_Volume')
        df_field = current_exchange_name  + '_obv'
        v_data_features[df_field] = df.loc[:,'obv'].values

        # compute exponential moving average of 5 weeks (trend) for OBV
        period = 5
        df_field = current_exchange_name  + '_obv_ema' + str(period)
        v_data_features[df_field] = v_data_features[current_exchange_name + '_obv'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
           
        # compute exponential moving average of 13 weeks (trend) for OBV
        period = 13
        df_field = current_exchange_name  + '_obv_ema' + str(period)
        v_data_features[df_field] = v_data_features[current_exchange_name + '_obv'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
           
        # compute exponential moving average of 26 weeks (trend) for OBV
        period = 26
        df_field = current_exchange_name  + '_obv_ema' + str(period)
        v_data_features[df_field] = v_data_features[current_exchange_name + '_obv'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
        
        # compute exponential moving average of 52 weeks (trend) for OBV
        period = 52
        df_field = current_exchange_name + '_obv_ema' + str(period)
        v_data_features[df_field] = v_data_features[current_exchange_name + '_obv'].ewm(ignore_na=False, min_periods=period, com=period, adjust=True).mean()
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
          

        # ----------------------------------------------------------------------
        # 52 Weeks High and Low

        # 52 Weeks High
        df_field = current_exchange_name  + '_52_Week_High'
        v_data_features[df_field] = np.log(v_data[current_exchange_name+'_Close'] / v_data[current_exchange_name+'_Close'].rolling(52).max())
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 

        # 52 Weeks Low
        df_field = current_exchange_name + '_52_Week_Low'
        v_data_features[df_field] = np.log(v_data[current_exchange_name+'_Close'] / v_data[current_exchange_name+'_Close'].rolling(52).min())
        # compute slope (first derivative)
        t_col1 = df_field
        period = 3
        order = 1  
        diff = slope(v_data_features, t_col=t_col1, lag=period)
        df_field = t_col1 + '_diff' + str(order) + '_period' + str(period)
        v_data_features[df_field] = diff 
    
        # stores each exchange DataFrame of features within a list 
        exchange_features_dataframe_list.append(v_data_features)
    
    return exchange_features_dataframe_list

In [93]:
files = ['CompleteWeeklyIndexes.csv','KNNWeeklyImputed.csv','MeanWeeklyImputed.csv', 'MIIWeeklyImputed.csv', 'MovingAverageWeeklyImputed.csv', 'RegressionWeeklyImputed.csv']
for file in files:
    print("Feature Engineering", file)
    df = read_csv(file)
    df_train = df.loc['2000-01-01':'2015-01-01']
    if 'Complete' in file:
        df_test = df.loc['2015-01-02':]
    else:
        test_data = file.split('.')
        df_test = read_csv(test_data[0] + '_test.' + test_data[1])
    
    # list of Exchange names
    print("Training data set")
    exc_name_list = ['sp500', 'vix', 'dji', 'ndx', 'n255', 'ftse', 'hsi', 'n100']
    temp_list_train = get_weekly_features_for_exchanges(df_train, exchange_name_list = exc_name_list)
    df_train_final = df_train
    for temp_df_train in temp_list_train:
        df_train_final = pd.concat([df_train_final, temp_df_train], axis=1, join='outer')
    output = file.split('.')
    df_train_final.to_csv(output[0] + '_train_FE.' + output[1], index = True)
    
    print("Test data set")
    temp_list_test = get_weekly_features_for_exchanges(df_test, exchange_name_list = exc_name_list)
    df_test_final = df_test
    for temp_df_test in temp_list_test:
        df_test_final = pd.concat([df_test_final, temp_df_test], axis=1, join='outer')
        
    df_test_final.to_csv(output[0] + '_test_FE.' + output[1], index = True)
    print("_______________________________________________________________")

Feature Engineering CompleteWeeklyIndexes.csv
Training data set
Feature Engineering for Exchange sp500
Feature Engineering for Exchange vix
Feature Engineering for Exchange dji
Feature Engineering for Exchange ndx
Feature Engineering for Exchange n255
Feature Engineering for Exchange ftse
Feature Engineering for Exchange hsi
Feature Engineering for Exchange n100
Test data set
Feature Engineering for Exchange sp500
Feature Engineering for Exchange vix
Feature Engineering for Exchange dji
Feature Engineering for Exchange ndx
Feature Engineering for Exchange n255
Feature Engineering for Exchange ftse
Feature Engineering for Exchange hsi
Feature Engineering for Exchange n100
_______________________________________________________________
Feature Engineering KNNWeeklyImputed.csv
Training data set
Feature Engineering for Exchange sp500
Feature Engineering for Exchange vix
Feature Engineering for Exchange dji
Feature Engineering for Exchange ndx
Feature Engineering for Exchange n255
Feature E

C:\ProgramData\Anaconda2\envs\mestrado\lib\site-packages\ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda2\envs\mestrado\lib\site-packages\ipykernel_launcher.py:331: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda2\envs\mestrado\lib\site-packages\ipykernel_launcher.py:342: RuntimeWarning: invalid value encountered in log


Feature Engineering for Exchange vix
Feature Engineering for Exchange dji
Feature Engineering for Exchange ndx
Feature Engineering for Exchange n255
Feature Engineering for Exchange ftse
Feature Engineering for Exchange hsi
Feature Engineering for Exchange n100
Test data set
Feature Engineering for Exchange sp500
Feature Engineering for Exchange vix
Feature Engineering for Exchange dji
Feature Engineering for Exchange ndx
Feature Engineering for Exchange n255
Feature Engineering for Exchange ftse
Feature Engineering for Exchange hsi
Feature Engineering for Exchange n100
_______________________________________________________________
Feature Engineering MovingAverageWeeklyImputed.csv
Training data set
Feature Engineering for Exchange sp500
Feature Engineering for Exchange vix
Feature Engineering for Exchange dji
Feature Engineering for Exchange ndx
Feature Engineering for Exchange n255
Feature Engineering for Exchange ftse
Feature Engineering for Exchange hsi
Feature Engineering for Exc